In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import sys 
import datetime as dt
import time
import gc
#gc.collect()


#http://www.imdb.com/interfaces/

In [2]:
from helper.parallel_util import ParallelUtil
from helper.parsers import Utils
from helper.url_utils import UrlUtils
from helper.file_mapping import FILE_MAPPINGS
from helper.imdb_sqllite_db import IMDBSQLLite
import config

LESSON_DATA_FOLDER = config.IMDB_DATA_FOLDER
NROWS = None
imdb_db = IMDBSQLLite();
imdbConn, imdbCurs = imdb_db.get()

IMDB SQLLite Database: ../data/imdb.sqllite


In [ ]:
#clusters = ParallelUtil('./data/parallel.config.tsv')      
#clusters.info()

#UrlUtils.get_file_name('https://datasets.imdbws.com/name.basics.tsv.gz')
#UrlUtils.is_downloadable('https://datasets.imdbws.com/name.basics.tsv.gz')




In [ ]:
#Step 1: Donwload latest imdb data files using config.IMDB_DATASETS

for url in config.IMDB_DATASETS:
    UrlUtils.download(url, config.IMDB_DATA_FOLDER)
    print (url)

In [ ]:
#Step 2: Load titles data
df_titles = pd.read_table('../data/title.basics.tsv.gz', nrows = NROWS)
df_titles.replace({'runtimeMinutes':{'\\N':0}, 'endYear':{'\\N':None}}, inplace= True, method='bfill')
#df_titles

In [ ]:
#Step 3: Load ratings data
df_ratings = pd.read_table('../data/title.ratings.tsv.gz', nrows = NROWS)
df_titles.replace({'\\N':None}, inplace= True, method='bfill')
df_ratings.dropna(axis=0, how='all', inplace = True)
#df_ratings

In [ ]:
#Step 4: Load episode data
df_episodes = pd.read_table('../data/title.episode.tsv.gz', nrows = NROWS)
df_episodes.replace({'\\N':None}, inplace= True, method='bfill')
df_episodes.dropna(axis=0, how='any', inplace = True)
#df_episodes.to_sql("title_episodes", imdbConn, if_exists="replace", index = False)

In [ ]:
#Step 5: Merge titles & ratings 
df_merged_title_ratings = df_titles.merge(df_ratings, on=['tconst'], how='left')
df_merged_title_ratings = df_merged_title_ratings.merge(df_episodes[['tconst','parentTconst']], on=['tconst'], how='left')

df_merged_title_ratings.replace({np.NaN:None}, inplace= True, method='bfill')
#df_merged_title_ratings
df_merged_title_ratings.to_sql("merged_title_ratings", imdbConn, if_exists="replace", index = False)

In [ ]:
#Step 6: Destroy titles, ratings & merged dataframes.
del [df_titles, df_ratings, df_merged_title_ratings, df_episodes]
gc.collect()

In [ ]:
#Step 7: Load names data
NROWS = None
df_names = pd.read_table('../data/name.basics.tsv.gz', nrows = NROWS)
df_names.replace({'\\N':None}, inplace= True, method='bfill')
#df_names

In [ ]:
#Step 8: Load names principals
df_principals = pd.read_table('../data/title.principals.tsv.gz', nrows = NROWS)
df_principals.replace({'\\N':None}, inplace= True, method='bfill')
#df_principals

In [ ]:
#Step 9: Merge names & principals 
df_merged = df_principals.merge(df_names, on=['nconst'], how='left')

df_merged.replace({np.NaN:None}, inplace= True, method='bfill')
#df_merged.head()
df_merged.to_sql("merged_name_principals", imdbConn, if_exists="replace", index = False)


In [ ]:
#Step 10: Destroy titles, ratings & merged dataframes.
del [df_names, df_principals, df_merged]
gc.collect()

In [ ]:
# Step 11: Drop & Create Indexes using config.IMDB_TABLE_DICT  
def drop_create_table_index(tableName):
    INDEX_DICT = config.IMDB_TABLE_DICT    
    indexes = INDEX_DICT[tableName]['indexs']
    for index in indexes:
        imdb_db.createIndex(tableName = tableName, indexName = index['indexName'], columns = index['columns'], 
                            isUnique = index['isUnique'])

drop_create_table_index('title_episodes')        
drop_create_table_index('merged_title_ratings')      
drop_create_table_index('merged_name_principals')        


In [ ]:
# Test 
print (imdb_db.rowCount('title_episodes'))    
print (imdb_db.rowCount('merged_title_ratings'))    
print (imdb_db.rowCount('merged_name_principals'))    


In [ ]:
# Any Random Test you may want to.
result = imdb_db.query('select  * from merged_title_ratings limit 100000');
result

In [3]:
imdbConn.execute("VACUUM")

OperationalError: database or disk is full